In [1]:
import pandas as pd
import numpy as np
import pymc3 as pm
print('Running on PyMC3 v{}'.format(pm.__version__))

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\Megan\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running on PyMC3 v3.5


In [2]:
df=pd.read_csv("201809-fordgobike-tripdata.csv")
df=df.dropna()
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip
0,76491,2018-09-30 18:37:01.7710,2018-10-01 15:51:53.5530,120.0,Mission Dolores Park,37.761420,-122.426435,26.0,1st St at Folsom St,37.787290,-122.394380,903,Customer,1998.0,Female,No
3,79396,2018-09-30 10:12:49.9400,2018-10-01 08:16:06.0440,11.0,Davis St at Jackson St,37.797280,-122.398436,55.0,Webster St at Grove St,37.777053,-122.429558,2458,Customer,1990.0,Male,No
5,70755,2018-09-30 12:26:06.6710,2018-10-01 08:05:22.5950,188.0,Dover St at 57th St,37.842630,-122.267738,190.0,West St at 40th St,37.830223,-122.270950,1483,Customer,1985.0,Male,No
6,57147,2018-09-30 15:06:12.7180,2018-10-01 06:58:40.4270,50.0,2nd St at Townsend St,37.780526,-122.390288,8.0,The Embarcadero at Vallejo St,37.799953,-122.398525,2706,Customer,1996.0,Male,No
7,20180,2018-09-30 20:13:33.6250,2018-10-01 01:49:53.6560,73.0,Pierce St at Haight St,37.771793,-122.433708,97.0,14th St at Mission St,37.768265,-122.420110,3979,Subscriber,1983.0,Female,No


In [3]:
df["duration_hrs"]=df["duration_sec"]/3600.
df["age"]=2019-df["member_birth_year"]
df.head()

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,bike_share_for_all_trip,duration_hrs,age
0,76491,2018-09-30 18:37:01.7710,2018-10-01 15:51:53.5530,120.0,Mission Dolores Park,37.761420,-122.426435,26.0,1st St at Folsom St,37.787290,-122.394380,903,Customer,1998.0,Female,No,21.247500,21.0
3,79396,2018-09-30 10:12:49.9400,2018-10-01 08:16:06.0440,11.0,Davis St at Jackson St,37.797280,-122.398436,55.0,Webster St at Grove St,37.777053,-122.429558,2458,Customer,1990.0,Male,No,22.054444,29.0
5,70755,2018-09-30 12:26:06.6710,2018-10-01 08:05:22.5950,188.0,Dover St at 57th St,37.842630,-122.267738,190.0,West St at 40th St,37.830223,-122.270950,1483,Customer,1985.0,Male,No,19.654167,34.0
6,57147,2018-09-30 15:06:12.7180,2018-10-01 06:58:40.4270,50.0,2nd St at Townsend St,37.780526,-122.390288,8.0,The Embarcadero at Vallejo St,37.799953,-122.398525,2706,Customer,1996.0,Male,No,15.874167,23.0
7,20180,2018-09-30 20:13:33.6250,2018-10-01 01:49:53.6560,73.0,Pierce St at Haight St,37.771793,-122.433708,97.0,14th St at Mission St,37.768265,-122.420110,3979,Subscriber,1983.0,Female,No,5.605556,36.0


In [4]:
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10, shape=1)
    sigma = pm.HalfNormal('sigma', sd=1)

    # Expected value of outcome
    mu = alpha + beta[0]*df['age']

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sigma, observed=df['duration_hrs'])

In [ ]:

map_estimate = pm.find_MAP(model=basic_model)

map_estimate

logp = -1.7044e+05, ||grad|| = 2.1735e+06:   0%| | 9/5000 [03:22<31:06:08, 22.43s/it]

In [ ]:
map_estimate = pm.find_MAP(model=basic_model, method='powell')

map_estimate

  0%|                                                 | 0/5000 [00:00<?, ?it/s]C:\Users\Megan\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:502: RuntimeWarning: Method powell does not use gradient information (jac).
  RuntimeWarning)
logp = -1.4541e+05, ||grad|| = 1.5338e+05:   2%| | 108/5000 [51:43<20:51:13, 15.35s/it]

In [ ]:
import seaborn as sbn
%matplotlib inline

sbn.stripplot(x="age",y="duration_hrs",data=df)
